In [ ]:
## developing a chatbot using opensource model gemma-2b
## using message history
# This chatbot will be able to have a conversation and remember previous interactions.

In [1]:
# to load env variables
import os
from dotenv import load_dotenv
load_dotenv()


True

In [14]:
# setting up goq api and model
from langchain_groq import ChatGroq
groq_api = os.getenv('GROQ_API_KEY')
llm_1 = ChatGroq(model='Gemma2-9b-It',groq_api_key=groq_api)
llm_1

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017E996A6210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017E9964D210>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
# here AImesaage is used to store the ai message given by model itself to store the info, if not given then model can,t remember the previous given output
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage

llm_1.invoke([HumanMessage(content="Hi my name is nithin and I am a student")])

AIMessage(content="Hi Nithin! It's nice to meet you. \n\nWhat kind of student are you? What are you studying? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 20, 'total_tokens': 52, 'completion_time': 0.058181818, 'prompt_time': 7.858e-05, 'queue_time': 0.013484859, 'total_time': 0.058260398}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3579c217-d58f-4466-b38f-e4b9ac86a250-0', usage_metadata={'input_tokens': 20, 'output_tokens': 32, 'total_tokens': 52})

In [19]:
# above we got an output in AIMessage now, we need to feed the model with same message to remember
messages = (
    [
        HumanMessage(content="Hi my name is nithin and I am a student"),
        AIMessage(content="Hi Nithin! It's nice to meet you. \n\nWhat kind of student are you? What are you studying?"), # feeding model with ai message
        HumanMessage(content="what is my name and who am I?") # checking weather model remembers my name
    ])

llm_1.invoke(messages)
# we cannot feed all the time to model so we use message history

AIMessage(content="You are Nithin, a student!  \n\nIt seems like you're curious about yourself.  \n\nWhat else would you like to know about who you are?  Perhaps you could tell me about your hobbies, your favorite subjects, or your dreams for the future. 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 65, 'total_tokens': 126, 'completion_time': 0.110909091, 'prompt_time': 0.001971253, 'queue_time': 0.011430515999999998, 'total_time': 0.112880344}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-44765ff7-e44f-4438-9df0-593564d6c829-0', usage_metadata={'input_tokens': 65, 'output_tokens': 61, 'total_tokens': 126})

### Message History
#### This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [23]:
from langchain_core.chat_history import BaseChatMessageHistory,InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory  # Stores messages in an in memory list.
from langchain_core.runnables.history import RunnableWithMessageHistory

store_sessions ={} # dict to store the sessions

# func to take session id and give the message history of that session
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store_sessions:
        store_sessions[session_id] = ChatMessageHistory()
    return store_sessions[session_id]

# runnaable with message history is used to interact with the llm model
with_message_history = RunnableWithMessageHistory(llm_1,get_session_history)


In [31]:
# We now need to create a config that we pass into the runnable every time
config = {"configurable": {"session_id": "chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is nithin and I an a DataScientist")],
    config = config)

response.content

"Hi Nithin! It's nice to meet you.  \n\nWhat kind of data science work do you enjoy doing?  \n\nI'm still learning about the field, so I'm always interested to hear about what people are working on. 😊 \n\n"

In [32]:
# checking the model
with_message_history.invoke(
    [HumanMessage(content="WHAT is my name?")],
    config = config
)

AIMessage(content='Your name is Nithin.  \n\nI remember that from our earlier conversation.  How can I help you today? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 90, 'total_tokens': 119, 'completion_time': 0.052727273, 'prompt_time': 0.002417722, 'queue_time': 0.012188967, 'total_time': 0.055144995}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-37f832fb-ed84-4f31-b5f6-9b5d85d63e5e-0', usage_metadata={'input_tokens': 90, 'output_tokens': 29, 'total_tokens': 119})

In [30]:
# if we change the config(session_id) and try to get the same
config1 ={'configurable':{"session_id":"chat2"}}

response_2  = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config = config1
)
response_2.content

# as new session is created model doesnt have previous session data
# we need to again feed the model with my name in diff session id and then try to get my name again

"I don't know your name.  \n\nI'm a large language model, so I don't have any memory of past conversations. If you'd like to tell me your name, I'd be happy to learn it! 😊 \n"

#### Using Prompt Template

In [40]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder 
# message place holder to pass all the messages in.

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful assitant. answer all the questions to your best ability"),
        MessagesPlaceholder(variable_name="messages") # what ever human messsage we want to give we have to give it to key-value pair with key="messages"
    ])

chain = prompt | llm_1


In [41]:
# after chaining to get the output

chain.invoke({"messages":[HumanMessage(content="Hi my name is nithin")]})

AIMessage(content="Hello Nithin! It's nice to meet you. \n\nIs there anything I can help you with today?  I'm ready for your questions! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 29, 'total_tokens': 67, 'completion_time': 0.069090909, 'prompt_time': 0.00023236, 'queue_time': 0.012832860000000001, 'total_time': 0.069323269}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2750aaa9-7174-48d5-aabe-ffed3b758145-0', usage_metadata={'input_tokens': 29, 'output_tokens': 38, 'total_tokens': 67})

In [44]:
# We can now wrap this in the same Messages History object as before
with_message_history_2 = RunnableWithMessageHistory(chain,get_session_history)
config2 = {"configurable":{"session_id":"chat3"}}
response_3 = with_message_history_2.invoke(
    [HumanMessage(content="Hi my name is Nithin")],
    config = config2)
response_3.content

"Hi Nithin!  It's nice to finally meet you.  \n\nHow can I be helpful today? 😊  \n\n"

In [45]:
# to get the output from above config
with_message_history_2.invoke([HumanMessage(content="What is my name")],config=config2)

AIMessage(content='Your name is Nithin!  😊 \n\nIs there anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 154, 'total_tokens': 178, 'completion_time': 0.043636364, 'prompt_time': 0.005883175, 'queue_time': 0.008483972, 'total_time': 0.049519539}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-230a9a8f-3bc9-4d16-8981-e841e942ce49-0', usage_metadata={'input_tokens': 154, 'output_tokens': 24, 'total_tokens': 178})

In [61]:
# we can complicate the prompt as well by adding extra variables
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful assitant. answer all the questions to your best ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain2 = prompt2 | llm_1

chain2.invoke({"messages":[HumanMessage(content="Hi my name is nithin")],"language":"Telugu"})

AIMessage(content='మీ పేరు నితిన్ అని తెలుసుకున్నాను! 😊   \n\nనేను మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాను. ఏదైనా ప్రశ్న ఉంటే, నేను నా శక్తికి సరిపడా జవాబు ఇస్తాను. \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 31, 'total_tokens': 110, 'completion_time': 0.143636364, 'prompt_time': 0.000369088, 'queue_time': 0.014875311, 'total_time': 0.144005452}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b4a271b3-c33b-43a3-a4da-98104d06ae86-0', usage_metadata={'input_tokens': 31, 'output_tokens': 79, 'total_tokens': 110})

In [65]:
# as we have multiple keys in input(messages,language) we are specifing the messages key
with_message_history_3 = RunnableWithMessageHistory(chain2,get_session_history,input_messages_key="messages")
config3 = {"configurable":{"session_id":"chat4"}}

response_4 = with_message_history_3.invoke({"messages":[HumanMessage(content="my name is nithinsai")],"language":"Hindi"},config=config3)
response_4.content

"That's a great name, Nithinsai!  😊  It's nice to officially know your full name.  \n\nIs there anything else I can help you with?\n"

### Managing Converstion History
#### One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

In [128]:
from langchain_core.messages import trim_messages,SystemMessage
#  trim_messages helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep

trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=llm_1, # counter is llm model
    include_system=True, # include the system message
    allow_partial=False,
    start_on="human",
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Nithin"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="I love reading fiction books")
]

trimmer.invoke(messages)

# here the trimmer only considered 40 tokens from last so, except sys message: first 4 messages are trimmed

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm Nithin", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love reading fiction books', additional_kwargs={}, response_metadata={})]

In [71]:
## Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.
# To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt


In [131]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# Runnable to passthrough inputs unchanged or with additional keys.
# rpt.assign is to add or modify data in the input dictionary by specifying keyword arguments.

chain3 = (RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer) | prompt2 | llm_1)

chain3.invoke({"messages":messages+[HumanMessage(content="what is my name?")],
               "language":"English"})

# in above although i have given less tokens name will get trimmed and model is not able to capture.

AIMessage(content='Your name is Nithin 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 133, 'total_tokens': 143, 'completion_time': 0.018181818, 'prompt_time': 0.00528705, 'queue_time': 0.007858489, 'total_time': 0.023468868}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ab9c9501-e201-452b-a68b-78181e087f3d-0', usage_metadata={'input_tokens': 133, 'output_tokens': 10, 'total_tokens': 143})

In [132]:
 # if we ask any question from untrimmed messages model should give output
chain3.invoke({"messages":messages+[HumanMessage(content="what math problem did i ask")],
               "language":"English"})


AIMessage(content='You asked "What\'s 2 + 2?"  😊 \n\nDo you have another one for me? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 115, 'total_tokens': 143, 'completion_time': 0.050909091, 'prompt_time': 0.003798866, 'queue_time': 0.011754673, 'total_time': 0.054707957}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-57a2d3c5-f469-4fe2-9487-db35ce337d61-0', usage_metadata={'input_tokens': 115, 'output_tokens': 28, 'total_tokens': 143})

In [133]:
#  wrap this in the Message History
with_message_history_4 = RunnableWithMessageHistory(chain3,get_session_history,input_messages_key="messages")

config4 = {"configurable": {"session_id": "chat5"}}

response_5 = with_message_history_4.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "English",
    },
    config=config4,
)

response.content
# name got trimmed

"Hi Nithin! It's nice to meet you.  \n\nWhat kind of data science work do you enjoy doing?  \n\nI'm still learning about the field, so I'm always interested to hear about what people are working on. 😊 \n\n"

In [136]:
with_message_history_4.invoke(
    {
        "messages": [HumanMessage(content="what do I love?")],
        "language": "English",
    },
    config=config4,
)
# model still have info about last human messages
# if you re-run this cell again and the last info is also gone


AIMessage(content="It's great to meet you! As an AI, I don't have personal feelings or experiences like love.  \n\nTo figure out what *you* love, maybe think about:\n\n* **What makes you feel happy and fulfilled?**\n* **What do you enjoy spending your time doing?**\n* **What are you passionate about?**\n* **What are you naturally good at?**\n\nExploring these questions can help you discover what brings you joy and makes you feel truly alive! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 37, 'total_tokens': 147, 'completion_time': 0.2, 'prompt_time': 0.000356499, 'queue_time': 0.01395501, 'total_time': 0.200356499}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5785345b-cc1e-48cb-886f-4dfe41edf651-0', usage_metadata={'input_tokens': 37, 'output_tokens': 110, 'total_tokens': 147})